In [1]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "Generate a concise story idea for a {genre} book."

llm = OpenAI(temperature=0)  # type: ignore
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
result = llm_chain("sci-fi")
print(result)

{'genre': 'sci-fi', 'text': '\n\nA group of scientists discover a mysterious alien artifact on the dark side of the moon. When they investigate, they find that the artifact is a gateway to a parallel universe, and they must travel through it to save their own world from an impending disaster.'}


In [2]:
result["text"]

'\n\nA group of scientists discover a mysterious alien artifact on the dark side of the moon. When they investigate, they find that the artifact is a gateway to a parallel universe, and they must travel through it to save their own world from an impending disaster.'

---


In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)

In [4]:
human_message_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        template="""I want you to brainstorm ideas for characters for my story. The genre is {genre}.
        Rank the ideas from most important to least important.
        
        Example response:
        1. Idea 1 ...,
        2. Idea 2 ...,
        3. Idea 3 ...
        
        """,
        input_variables=["genre"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)  # type: ignore
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

chain.run({"genre": "fantasy"})
# chain("fantasy") # This will work too using the __call__ method

"1. Main Protagonist - A young orphan with a hidden magical ability who embarks on a quest to save the kingdom from an evil sorcerer.\n2. Main Antagonist - The evil sorcerer who seeks to gain ultimate power and control over the kingdom by harnessing dark magic.\n3. Mentor - A wise and powerful wizard who aids the protagonist on their journey, teaching them how to control and master their magical abilities.\n4. Sidekick - A loyal and comical creature, such as a talking animal or mischievous imp, who provides comic relief and assists the protagonist in their quest.\n5. Love Interest - A strong and independent individual with their own magical abilities, who becomes a romantic interest for the protagonist and aids them in their quest.\n6. Rival - A fellow magical apprentice who initially competes with the protagonist but eventually becomes an ally, adding an element of rivalry and growth to the story.\n7. Supporting Characters - Various characters who assist or hinder the protagonist thro

In [5]:
genres = [
    {"genre": "fantasy"},
    {"genre": "sci-fi"},
]

# Generate multiple LLM requests based on multiple inputs:
# The async method is .aapply()
print(chain.apply(genres))

[{'text': "1. Idea 1: The protagonist - a young orphan with a mysterious past who discovers they have magical abilities and is thrust into an epic quest to save their kingdom.\n2. Idea 2: The mentor - an ancient, wise wizard who guides the protagonist on their journey and helps them harness their magical powers.\n3. Idea 3: The antagonist - a power-hungry sorcerer who seeks to take over the kingdom and will stop at nothing to eliminate anyone who stands in their way.\n4. Idea 4: The sidekick - a loyal and humorous creature or magical being who provides comic relief and assists the protagonist in their adventures.\n5. Idea 5: The love interest - a fierce and independent warrior or a fellow magic user who joins the protagonist on their quest and eventually falls in love with them.\n6. Idea 6: The rogue - a skilled thief or assassin who initially opposes the protagonist but later becomes an ally, adding an element of unpredictability and deception.\n7. Idea 7: The wise elder - an ancient 

In [6]:
# The async method is .agenerate()
chain.generate(genres)

LLMResult(generations=[[ChatGeneration(text="1. Idea 1: The protagonist - a young orphan with a mysterious past who discovers they have unique and powerful magical abilities. This character will drive the plot and embark on a journey to uncover their true identity and fulfill their destiny.\n\n2. Idea 2: The mentor - an experienced and wise wizard who becomes the protagonist's guide and teacher. This character will provide guidance, impart knowledge, and help the protagonist navigate the magical world they are entering.\n\n3. Idea 3: The antagonist - a power-hungry sorcerer who seeks to harness dark magic and conquer the realm. This character will create conflict and obstacles for the protagonist, serving as a formidable adversary throughout the story.\n\n4. Idea 4: The loyal companion - a talking animal or mythical creature who befriends the protagonist and accompanies them on their journey. This character will provide companionship, comic relief, and assist the protagonist with their

Analyzing issues within a Chain object by its output alone can prove to be a daunting task, primarily because such objects often undergo substantial transformation—both before the input prompt is processed and after the LLM output is generated.

However, a solution lies within your reach. By setting the `verbose` parameter to `True`, you are enabling a feature that unveils the internal dynamics of the Chain object during its runtime. This provision can offer you invaluable insights, further empowering you to troubleshoot and resolve any underlying issues more effectively.


In [7]:
# You can debug all LLMChains by turning on Verbose=True:
chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=True)
chain.run({"genre": "fantasy"})



> Entering new  chain...
Prompt after formatting:
System: I want you to brainstorm ideas for characters for my story. The genre is fantasy.
        Rank the ideas from most important to least important.
        
        Example response:
        1. Idea 1 ...,
        2. Idea 2 ...,
        3. Idea 3 ...
        
        

> Finished chain.


"1. Main protagonist - a young orphan with a mysterious past who discovers they have a unique magical ability and is chosen by an ancient prophecy to save the world.\n2. Wise mentor - an elderly wizard who guides and trains the protagonist in their magical abilities and imparts wisdom and knowledge about the world of magic.\n3. Antagonist - a power-hungry sorcerer who seeks to harness dark magic to gain ultimate control and is willing to stop at nothing to achieve his goals.\n4. Sidekick - a loyal and mischievous magical creature who befriends the protagonist and provides comic relief as well as assistance in their quest.\n5. Love interest - a skilled warrior from a different kingdom, who initially clashes with the protagonist but eventually falls in love and joins their cause.\n6. Supporting character - a skilled healer or herbalist who possesses knowledge of ancient remedies and mystical potions that aid the protagonist in their journey.\n7. Villain's henchman - a loyal and fearless 

LLMChain does not automatically parse the output by default, even if the prompt object has an output parser. To apply the output parser on the LLM output you can add an output_parser.


In [8]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [9]:
class Ideas(BaseModel):
    ideas: List[str] = Field(description="A list of ideas for a story.")


parser = PydanticOutputParser(pydantic_object=Ideas)

prompt = PromptTemplate(
    template="I want you to brainstorm ideas for characters for my story. The genre is {genre}.\n  {format_instructions}",
    input_variables=["genre", "format_instructions"],
)

chain = LLMChain(llm=chat, prompt=prompt, output_parser=parser)

In [10]:
result = chain.run(
    {"genre": "fantasy", "format_instructions": parser.get_format_instructions()}
)

In [14]:
print(result)
print(result.dict())  # type: ignore

ideas=['A powerful sorcerer who has lost their memory and must journey to reclaim their past', 'A street thief with the ability to control shadows', 'A young orphan who discovers they are the chosen one destined to save the world', 'A wise and ancient dragon who serves as a mentor to the protagonist', 'A mischievous fairy who loves playing pranks on humans', 'A skilled warrior who wields a magical sword', 'A shape-shifting creature who can take the form of any animal', 'A cursed princess trapped in a tower awaiting rescue', 'A group of mystical creatures who band together to form a secret society', 'A mysterious wizard who lives in an enchanted forest', 'A talking animal companion who provides comic relief', 'A vengeful ghost seeking revenge against their murderer', 'A young wizard apprentice with untapped and unpredictable powers', 'A noble knight on a quest to find the legendary holy grail', 'An ancient prophecy foretelling the rise of a dark evil and the chosen hero who will defeat 

---


Given that your story generation will require multiple sequential prompts you can use a `SequentialChain` to chain multiple prompts together. The `SequentialChain` will automatically re-use the output of the previous prompt and use it as the input for the next prompt.


In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain

In [47]:
character_generation_prompt = PromptTemplate(
    template="""I want you to brainstorm 3 - 5 characters for my short story. The genre is {genre}.
    Each character must have a Name and a Biography.
    You must provide a name and biography for each character, this is very important!
    ---
    Example response:
    Name: CharWiz, Biography: A wizard who is a master of magic.
    Name: CharWar, Biography: A warrior who is a master of the sword.
    ---

    Characters: """,
    input_variables=["genre"],
)

plot_generation_prompt = PromptTemplate(
    template="""Given the following characters and the genre, create an effective plot for a short story:
    Characters:
    {characters} 
    ---
    Genre: {genre}
    ---
    Plot: """,
    input_variables=["genre", "characters"],
)

scene_generation_plot = PromptTemplate(
    template="""Act as an effective content creator. 
    Given multiple characters and a plot you are responsible generating the various scenes for each act. 
    
    You must de-compose the plot into multiple effective scenes:

    ---
    Characters:
    {characters}
    ---
    Genre: {genre}
    ---
    Plot: {plot}
    ---
    Example response:
    Scenes:
    Scene 1: Some text here.
    Scene 2: Some text here.
    Scene 3: Some text here.
    ----
    Scenes:
    """,
    input_variables=["genre", "characters", "plot"],
)

In [52]:
chat_model = ChatOpenAI()  # type: ignore
character_generation_chain = LLMChain(
    llm=chat, prompt=character_generation_prompt, output_key="characters"
)
plot_generation_chain = LLMChain(
    llm=chat, prompt=plot_generation_prompt, output_key="plot"
)
scene_generation_chain = LLMChain(
    llm=chat, prompt=scene_generation_plot, output_key="scenes"
)

In [53]:
story_chain = SequentialChain(
    chains=[
        character_generation_chain,
        plot_generation_chain,
        scene_generation_chain,
    ],
    input_variables=["genre", "format_instructions"],
    output_variables=["characters", "plot", "scenes", "genre"],
)

In [54]:
story_result = story_chain({"genre": "fantasy", "format_instructions": ""})

In [57]:
print(story_result)

{'genre': 'fantasy', 'format_instructions': '', 'characters': 'Name: Isabella Nightshade\nBiography: Isabella Nightshade is a skilled and mysterious sorceress who possesses the power to manipulate shadows and darkness. Raised in the enigmatic Shadowrealm, she has honed her abilities over many years, making her a formidable force to be reckoned with. Isabella is known for her calm demeanor and calculated tactics, often using her powers to stealthily infiltrate enemy territories and disable her opponents with deadly precision.\n\nName: Orion Stormrider\nBiography: Orion Stormrider is a seasoned elven archer with unparalleled accuracy and agility. Growing up in the ancient forest of Eldoria, he learned the art of archery from the renowned elven elders. Orion is known for his unwavering focus and stoic nature, always keeping his cool even in the midst of chaos. His arrows are said to be guided by the spirits of the forest, and his aim is rarely, if ever, off the mark.\n\nName: Seraphina Fr

---


## Sequential Story Scene Generation:


In [94]:
from langchain.memory import SimpleMemory

# Extracting the scenes using .split('\n') and removing empty strings:
scenes = [scene for scene in story_result["scenes"].split("\n") if scene]

character_script_prompt = PromptTemplate(
    template="""Given the following characters: {characters} and the genre: {genre}, create an effective character script for a scene.

    You must follow the following principles:
    - Use the Previous Scene Summary: {previous_scene_summary} to avoid repeating yourself.
    - Use the Plot: {plot} to create an effective scene character script.
    - Currently you are generating the character dialogue script for the following scene: {scene}

    ---
    Here is an example response:
    SCENE 1: ANNA'S APARTMENT

    (ANNA is sorting through old books when there is a knock at the door. She opens it to reveal JOHN.)
    ANNA: Can I help you, sir?
    JOHN: Perhaps, I think it's me who can help you. I heard you're researching time travel.
    (Anna looks intrigued but also cautious.)
    ANNA: That's right, but how do you know?
    JOHN: You could say... I'm a primary source.

    ---
    SCENE {index}:

    """,
    input_variables=[
        "characters",
        "genre",
        "plot",
        "scene",
        "previous_scene_summary",
        "index",
    ],
)

summarize_prompt = PromptTemplate(
    template="""Given a character script create a summary of the scene. Character script: {character_script}""",
    input_variables=["character_script"],
)

character_script_chain = LLMChain(
    llm=chat, prompt=character_script_prompt, output_key="character_script"
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt, output_key="summary")
previous_scene_summary = ""

# Creating a simple memory to store the plot, characters, and genre:
memory = SimpleMemory(
    memories={
        "plot": story_result["plot"],
        "characters": story_result["characters"],
        "genre": "fantasy",
    }
)

generated_scenes = []

for index, scene in enumerate(scenes):
    print(f"Generating scene {index + 1}...")
    scene_generation_chain = SequentialChain(
        chains=[character_script_chain, summarize_chain],
        memory=memory,
        input_variables=[
            "scene",
            "previous_scene_summary",
            "index",
        ],
        verbose=True,
        output_variables=["character_script", "summary"],
    )

    llm_result = scene_generation_chain(
        {
            "characters": story_result["characters"],
            "genre": "fantasy",
            "plot": story_result["plot"],
            "scene": scene,
            "previous_scene_summary": previous_scene_summary,
            "index": index + 1,
        }
    )

    # Updating the previous scene summary:
    previous_scene_summary = llm_result["summary"]

    # Store the generated scenes:
    generated_scenes.append(
        {"character_script": llm_result["character_script"], "scene": scenes[index]}
    )

Generating scene 1...


> Entering new  chain...

> Finished chain.
Generating scene 2...


> Entering new  chain...

> Finished chain.
Generating scene 3...


> Entering new  chain...

> Finished chain.
Generating scene 4...


> Entering new  chain...

> Finished chain.
Generating scene 5...


> Entering new  chain...

> Finished chain.
Generating scene 6...


> Entering new  chain...

> Finished chain.
Generating scene 7...


> Entering new  chain...

> Finished chain.
Generating scene 8...


> Entering new  chain...

> Finished chain.
Generating scene 9...


> Entering new  chain...

> Finished chain.
Generating scene 10...


> Entering new  chain...

> Finished chain.


In [101]:
generated_scenes

[{'character_script': 'IN ISABELLA\'S STUDY - NIGHT\n\n(The room is dimly lit, with shelves filled with ancient tomes and artifacts. Isabella sits at her desk, studying an old parchment with a series of enigmatic symbols and writings. She furrows her brows, deep in concentration.)\n\nISABELLA: (muttering to herself) "In the heart of Eldoria lies the key, a relic of ancient power, hidden from sight. Seek the moonlit grove and the whispering trees, where shadows dance in the pale moon\'s light."\n\n(She traces her finger along the parchment, following the cryptic clues. Suddenly, the room darkens, and a faint whisper echoes through the air.)\n\nMYSTERIOUS VOICE: (whispering) Isabella Nightshade, the time has come. The Shadowrealm beckons you.\n\n(Isabella looks around, searching for the source of the voice.)\n\nISABELLA: Who\'s there? Show yourself!\n\n(The shadows in the room seem to come alive, swirling around Isabella. She gazes into the darkness, her eyes shining with determination.)

In [141]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import pandas as pd

In [142]:
df = pd.DataFrame(generated_scenes)

In [143]:
all_character_script_text = "\n".join(df.character_script.tolist())

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=200
)

docs = text_splitter.create_documents([all_character_script_text])

In [140]:
chain = load_summarize_chain(llm=chat, chain_type="map_reduce")
summary = chain.run(docs)
print(summary)

'Isabella Nightshade, Seraphina Frostheart, and Orion Stormrider form an alliance to find an ancient artifact and save their realm from darkness. They utilize their unique abilities to overcome challenges, defeat enemies, and claim the artifact. In a final battle against darkness, they unite their powers and bring light back to their realm. They part ways but remain connected by their shared experiences.'